# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import tensorflow as tf
from sklearn.preprocessing import StandardScaler

import vertexai
from vertexai.generative_models import GenerativeModel, Part
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform_v1beta1 as aiplatform
import uuid

from tqdm import tqdm
import yfinance as yf
import lxml    

# Import Data

In [2]:
df_sandp500 = pd.read_csv('/Users/ani/Projects/6_stock_portfolio_recommendation/data/sp500_data_sample.csv')
df_sandp500 = df_sandp500.rename(columns={'Security': 'Company_Name'})
df_sandp500

,Ticker,Company_Name,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Annualized_Return,YTD_Pct_Return,...,2023_Pct_Return,2022_Pct_Return,2021_Pct_Return,2020_Pct_Return,Market_Cap,Pct_Diff_200_MA,Annualized_Volatility,Sharpe_Ratio,Beta,Years_Since_Founded
0,MSFT,Microsoft,Information Technology,Systems Software,"Redmond, Washington",1994-06-01,789019,1975,21.32,10.40,...,58.35,-27.69,55.79,22.27,3.421644e+12,10.81,0.27,78.51,1.18,50.0
1,NVDA,Nvidia,Information Technology,Semiconductors,"Santa Clara, California",2001-11-30,1045810,1993,72.79,-2.29,...,246.10,-51.44,124.48,48.40,3.295497e+12,6.61,0.53,137.30,2.07,32.0
2,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30,320193,1977,20.76,-17.44,...,54.80,-28.20,38.06,65.49,2.999856e+12,-10.79,0.30,69.06,1.27,48.0
3,AMZN,Amazon,Consumer Discretionary,Broadline Retail,"Seattle, Washington",2005-11-18,1018724,1994,10.66,-6.91,...,77.04,-50.71,4.64,31.80,2.176468e+12,1.80,0.36,29.80,1.43,31.0
4,GOOG,Alphabet Inc. (Class C),Communication Services,Interactive Media & Services,"Mountain View, California",2006-04-03,1652044,1998,19.39,-9.22,...,57.11,-38.84,67.43,22.35,2.090085e+12,0.16,0.31,62.44,1.24,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,APA,APA Corporation,Energy,Oil & Gas Exploration & Production,"Houston, Texas",1997-07-28,1841666,1954,10.16,-25.26,...,-15.71,69.08,83.27,21.51,6.137973e+09,-20.98,0.55,18.30,1.39,71.0
499,CZR,Caesars Entertainment,Consumer Discretionary,Casinos & Gaming,"Reno, Nevada",2021-03-22,1590895,1973,-5.72,-17.52,...,10.93,-55.49,30.63,105.79,5.590180e+09,-22.33,0.56,-10.32,1.97,52.0
500,ENPH,Enphase Energy,Information Technology,Semiconductor Materials & Equipment,"Fremont, California",2021-01-07,1463101,2006,-7.06,-42.00,...,-47.83,43.65,6.21,194.02,5.430658e+09,-44.18,0.68,-10.33,1.48,19.0
501,BRK.B,Berkshire Hathaway,Financials,Multi-Sector Holdings,"Omaha, Nebraska",2010-02-16,1067983,1839,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
VECTOR_SEARCH_INDEX_ID="projects/307012892661/locations/us-central1/indexEndpoints/240979963459665920"
DEPLOYED_INDEX_ID="stock_recommendation_app_1749700932103"
PROJECT_NUMBER = "307012892661"

In [ ]:
import pandas as pd
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from vertexai.language_models import TextEmbeddingModel
from google.cloud import aiplatform_v1 as aiplatform 
from google.cloud.aiplatform_v1.types import UpsertDatapointsRequest, IndexDatapoint, FindNeighborsRequest
import numpy as np
import uuid
import os 

# --- Google Cloud Configuration ---
# Replace with your Google Cloud Project ID and Region
PROJECT_ID = "capable-arbor-293714"
LOCATION = "us-central1" # Or your preferred region for Vertex AI
# IMPORTANT: Replace with your actual Google Cloud Project Number
# You can find this in the GCP Console under "Project info" or "IAM & Admin" -> "Settings".
PROJECT_NUMBER = "307012892661"

# --- Vertex AI Vector Search Configuration ---
# IMPORTANT: Replace with your actual Vertex AI Vector Search Index ID and Deployed Index ID
# You need to have created and deployed a Vector Search index in your project.
VECTOR_SEARCH_INDEX_ID="2810072444642000896"
ENDPOINT_ID="240979963459665920"

# Dynamically construct the specific API endpoint for the MatchServiceClient (for querying)
# This format uses the Project Number for the VDB endpoint.
# API_ENDPOINT_FOR_MATCH_CLIENT = f"{ENDPOINT_ID}.{LOCATION}-{PROJECT_NUMBER}.vdb.vertexai.goog"
API_ENDPOINT_FOR_MATCH_CLIENT="801674555.us-central1-307012892661.vdb.vertexai.goog"

# Initialize Vertex AI
try:
    vertexai.init(project=PROJECT_ID, location=LOCATION)
    print(f"Vertex AI initialized for project '{PROJECT_ID}' in location '{LOCATION}'.")
except Exception as e:
    print(f"ERROR: Failed to initialize Vertex AI. Please ensure your project ID and location are correct and you are authenticated. Error: {e}")
    exit() 

# Initialize Vertex AI models
try:
    # Using "gemini-2.0-flash". If you continue to get 404 errors,
    # please verify the exact model name and its availability in your region
    # (e.g., "gemini-pro", "gemini-1.5-pro-001") via Google Cloud documentation.
    generation_model = GenerativeModel("gemini-2.0-flash") 
    embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")
    print("Vertex AI Generative and Embedding models loaded.")
except Exception as e:
    print(f"ERROR: Failed to load Vertex AI models. Error: {e}")
    exit()

# Initialize Vertex AI Vector Search clients
try:
    # Client for managing and uploading data to the index (uses standard regional endpoint)
    index_service_client = aiplatform.IndexServiceClient(client_options={"api_endpoint": f"{LOCATION}-aiplatform.googleapis.com"})
    # Client for querying the deployed index (uses the specific VDB endpoint)
    match_service_client = aiplatform.MatchServiceClient(client_options={"api_endpoint": API_ENDPOINT_FOR_MATCH_CLIENT})
    print("Vertex AI Vector Search clients initialized.")
except Exception as e:
    print(f"ERROR: Failed to initialize Vertex AI Vector Search clients. Error: {e}")
    exit()


# --- RAG Components ---

def get_langchain_text_chunks(text, chunk_size=1000, chunk_overlap=100):
    """
    Splits text into overlapping chunks using LangChain's RecursiveCharacterTextSplitter.
    """
    if not text:
        return []
    
    # Create a LangChain Document object from the text
    document = Document(page_content=text)
    
    # Initialize the text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len, # Use standard len for character count
        add_start_index=True,
    )
    
    # Split the document
    chunks = text_splitter.split_documents([document])
    
    # Return the content of the chunks
    return [chunk.page_content for chunk in chunks]


def process_dataframe_for_vector_search(df: pd.DataFrame, text_columns: list):
    """
    Processes a Pandas DataFrame, creates textual chunks from specified columns using LangChain,
    generates embeddings, and upserts them to Vertex AI Vector Search.

    Args:
        df (pd.DataFrame): The input DataFrame (e.g., df_sandp500).
        text_columns (list): A list of column names whose content should be combined
                             to form the text for embedding.
    Returns:
        dict: An in-memory dictionary storing text chunks keyed by their unique IDs.
    """
    # Initialize document_store locally within this function
    document_store = {} 

    print(f"Processing DataFrame with {len(df)} rows and columns: {text_columns}")
    all_datapoints = []
    processed_count = 0

    for index, row in df.iterrows():
        # Combine specified columns into a single text string for RAG
        combined_text = " ".join(str(row[col]) for col in text_columns if col in row)
        
        # Use LangChain to get text chunks
        chunks = get_langchain_text_chunks(combined_text)
        if not chunks:
            print(f"WARNING: No text extracted or chunks created from row {index}. Skipping.")
            continue

        # Generate embeddings for the chunks (batching for efficiency)
        embeddings_for_row_chunks = []
        for i in range(0, len(chunks), 5): # Batch requests to avoid API limits
            batch_chunks = chunks[i:i+5]
            try:
                # Assuming get_embeddings returns a list of Embedding objects directly
                response = embedding_model.get_embeddings(batch_chunks)
                for j, emb_item in enumerate(response): 
                    embeddings_for_row_chunks.append((batch_chunks[j], emb_item.values))
            except Exception as e:
                print(f"ERROR: Error generating embeddings for a batch from row {index}: {e}. Skipping this batch.")
                continue

        if not embeddings_for_row_chunks:
            print(f"WARNING: No embeddings could be generated for row {index}. Skipping.")
            continue

        # Prepare IndexDatums for Vector Search and populate document_store
        for chunk, embedding_vector in embeddings_for_row_chunks:
            data_id = str(uuid.uuid4()) # Unique ID for each data point
            
            # Store the actual text chunk in our in-memory document store
            document_store[data_id] = chunk
            print(f"Storing chunk with ID {data_id} in document store.")

            datapoint = IndexDatapoint(
                datapoint_id=data_id,
                feature_vector=embedding_vector
            )
            all_datapoints.append(datapoint)
        processed_count += 1
        if processed_count % 100 == 0:
            print(f"Processed {processed_count} rows, generated {len(all_datapoints)} data points so far.")

    # Upsert all collected datapoints to Vertex AI Vector Search
    if not all_datapoints:
        print("No data points to upsert to Vector Search.")
        return document_store

    try:
        index_resource_name = index_service_client.index_path(PROJECT_ID, LOCATION, VECTOR_SEARCH_INDEX_ID)
        
        # Vertex AI Vector Search has limits on the number of datapoints per upsert call.
        # It's good practice to batch them if you have many.
        batch_size = 1000
        for i in range(0, len(all_datapoints), batch_size):
            batch_datapoints = all_datapoints[i:i + batch_size]
            print(f"Upserting batch {i//batch_size + 1}/{(len(all_datapoints) + batch_size - 1)//batch_size} ({len(batch_datapoints)} datapoints)...")
            
            # Construct the request object
            request = UpsertDatapointsRequest(
                index=index_resource_name,
                datapoints=batch_datapoints
            )
            
            # Pass the request object as a keyword argument
            response = index_service_client.upsert_datapoints(request=request)
            # Removed direct access to .name due to observed error; assuming success if no exception
            print(f"Batch upserted successfully.") 
        
        print(f"Successfully upserted a total of {len(all_datapoints)} data points to Vector Search.")
        print("Note: It may take some time for the changes to propagate and be searchable in Vector Search.")
    except Exception as e:
        print(f"ERROR: Error upserting data to Vertex AI Vector Search: {e}")
        print("Please ensure your Vector Search Index and Endpoint are correctly configured and deployed.")
    
    return document_store # Return the populated document_store


def retrieve_relevant_documents(query, document_store, top_k=3): # Added document_store as argument
    """
    Retrieves the most relevant document chunks from Vertex AI Vector Search.
    """
    try:
        query_embedding_response = embedding_model.get_embeddings([query])
        query_embedding = query_embedding_response[0].values 
    except Exception as e:
        print(f"ERROR: Error generating query embedding: {e}")
        return []

    try:
        # Construct the full index endpoint resource name (used for index_endpoint in FindNeighborsRequest)
        index_endpoint_resource_name = ( 
            f"projects/{PROJECT_NUMBER}/locations/{LOCATION}/indexEndpoints/{ENDPOINT_ID}"
        )

        # Build FindNeighborsRequest object as shown in the provided snippet
        datapoint = IndexDatapoint(
            feature_vector=query_embedding
        )

        query_obj = FindNeighborsRequest.Query(
            datapoint=datapoint,
            neighbor_count=top_k # The number of nearest neighbors to be retrieved
        )

        # Construct the FindNeighborsRequest with keyword arguments as expected by v1
        request = FindNeighborsRequest(
            index_endpoint=index_endpoint_resource_name,
            deployed_index_id="stock_recommendation_app_1749700932103",
            queries=[query_obj],
            return_full_datapoint=False,
        )

        # Execute the request
        response = match_service_client.find_neighbors(request=request)

        relevant_chunks = []
        print("\n--- Retrieved Neighbors (for debugging) ---")
        for neighbor in response.nearest_neighbors[0].neighbors:
            # Use the datapoint_id to retrieve the original text chunk from our document_store
            chunk_id = neighbor.datapoint.datapoint_id
            print(f"Chunk ID: {chunk_id}")
            chunk_text = document_store.get(chunk_id, "Error: Chunk text not found in store.")
            
            if chunk_text:
                relevant_chunks.append(chunk_text)
                print(f"  - Similarity: {neighbor.distance:.4f}, Chunk ID: {chunk_id}, Chunk: \"{chunk_text[:100]}...\"")
        print("-------------------------------------------\n")
        return relevant_chunks

    except Exception as e:
        print(f"ERROR: Error retrieving from Vertex AI Vector Search: {e}")
        print("Please ensure your Vector Search Index and Endpoint are correctly configured and deployed.")
        return []

def generate_response_with_rag(user_query, document_store): # Added document_store as argument
    """Generates a response using retrieved context and Gemini."""
    relevant_docs = retrieve_relevant_documents(user_query, document_store) # Passed document_store

    if not relevant_docs:
        print("WARNING: Could not find relevant documents in the knowledge base. Generating response based on general knowledge.")
        context_text = ""
    else:
        context_text = "\n\n".join(relevant_docs)
        print("\n--- Context provided to LLM ---")
        print(context_text)
        print("-------------------------------\n")

    # Construct the prompt with context
    prompt = f"""
    You are a helpful assistant. Answer the user's question truthfully based on the provided context.
    If the answer is not available in the context, state that you cannot find the answer in the provided information.

    Context:
    {context_text}

    Question: {user_query}

    Answer:
    """

    try:
        response = generation_model.generate_content(
            [Part.from_text(prompt)],
            generation_config={"temperature": 0.2}
        )
        return response.text
    except Exception as e:
        print(f"ERROR: Error generating content with Gemini: {e}")
        return "An error occurred while generating the response."

Vertex AI initialized for project 'capable-arbor-293714' in location 'us-central1'.
Vertex AI Generative and Embedding models loaded.


In [ ]:
# --- Example Usage in a Jupyter Notebook ---
print("--- Starting RAG Notebook Script ---")

# 1. Create a dummy DataFrame for demonstration
# In your actual notebook, df_sandp500 would be loaded from a file or database
data = {
    'Ticker': ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA'],
    'Company_Name': ['Apple Inc.', 'Microsoft Corp.', 'Alphabet Inc. (Class A)', 'Amazon.com Inc.', 'Tesla Inc.'],
    'Annualized_Return': [0.25, 0.30, 0.20, 0.18, 0.40],
    'Market_Cap_B': [3000, 2500, 2000, 1800, 800],
    'Sector': ['Technology', 'Technology', 'Communication Services', 'Consumer Discretionary', 'Consumer Discretionary'],
    'Description': [
        "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells related services.",
        "Microsoft Corporation develops, licenses, and supports software, services, devices, and solutions worldwide. Its segments include Productivity and Business Processes, Intelligent Cloud, and More Personal Computing.",
        "Alphabet Inc. provides various products and services worldwide. It operates through Google Services, Google Cloud, and Other Bets segments.",
        "Amazon.com, Inc. engages in the retail sale of consumer products and subscriptions in North America and internationally. It operates through North America, International, and Amazon Web Services (AWS) segments.",
        "Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally."
    ]
}
df_sandp500 = pd.DataFrame(data)
print("\nDummy df_sandp500 created:")
print(df_sandp500.head())

# 2. Process the DataFrame and upsert to Vector Search
# The document_store is now returned by this function
columns_to_embed = ['Ticker', 'Company_Name', 'Annualized_Return', 'Sector', 'Description']
# Store the returned document_store in a local variable
my_document_store = process_dataframe_for_vector_search(df_sandp500, columns_to_embed)

# Give some time for Vector Search changes to propagate if running immediately after upsert
print("\nWaiting a few seconds for Vector Search changes to propagate...")
import time
time.sleep(10) # Adjust as needed based on your index configuration

--- Starting RAG Notebook Script ---

Dummy df_sandp500 created:
  Ticker             Company_Name  Annualized_Return  Market_Cap_B  \
0   AAPL               Apple Inc.               0.25          3000   
1   MSFT          Microsoft Corp.               0.30          2500   
2  GOOGL  Alphabet Inc. (Class A)               0.20          2000   
3   AMZN          Amazon.com Inc.               0.18          1800   
4   TSLA               Tesla Inc.               0.40           800   

                   Sector                                        Description  
0              Technology  Apple Inc. designs, manufactures, and markets ...  
1              Technology  Microsoft Corporation develops, licenses, and ...  
2  Communication Services  Alphabet Inc. provides various products and se...  
3  Consumer Discretionary  Amazon.com, Inc. engages in the retail sale of...  
4  Consumer Discretionary  Tesla, Inc. designs, develops, manufactures, l...  
Processing DataFrame with 5 rows and col

In [42]:
# 3. Test the RAG system with queries
print("\n--- Testing RAG Queries ---")
queries = [
    "What is the annualized return for Apple?",
    "Tell me about Microsoft's main business segments.",
    "Which companies are in the Consumer Discretionary sector?",
    "What is the market cap of Google?", 
    "What is the capital of France?" 
]

for query in queries:
    print(f"\nUser Query: {query}")
    # Pass my_document_store to generate_response_with_rag
    response = generate_response_with_rag(query, my_document_store) 
    print(f"Assistant Response: {response}")
    print("-" * 50)

print("\n--- RAG Notebook Script Finished ---")



--- Testing RAG Queries ---

User Query: What is the annualized return for Apple?

--- Retrieved Neighbors (for debugging) ---
  - Similarity: 0.6347, Chunk ID: 673e4c20-883b-4b7b-8984-e26ce37468d4, Chunk: "Error: Chunk text not found in store...."
  - Similarity: 0.6347, Chunk ID: 553558fd-28b7-4254-8e88-0c56fe406c4b, Chunk: "Error: Chunk text not found in store...."
  - Similarity: 0.6347, Chunk ID: 09bfc957-1a7e-4619-8a6d-e8fa3df8da83, Chunk: "Error: Chunk text not found in store...."
-------------------------------------------


--- Context provided to LLM ---
Error: Chunk text not found in store.

Error: Chunk text not found in store.

Error: Chunk text not found in store.
-------------------------------

Assistant Response: I am unable to answer the question. The provided context contains only the error message "Error: Chunk text not found in store." and does not contain any information about Apple's annualized return.

--------------------------------------------------

User Qu